#### Lesson 6: Planning and Stock Report Generation

#### Setup

In [12]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;
using System.IO;
using Microsoft.SemanticKernel;
using System.ComponentModel;
using AutoGen.SemanticKernel;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-4o-mini";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.16

#### Note
This example is slightly different with the python ones when it comes to engineer agent

due to the lacking built-in support of running python code in C# AutoGen.

The engineer and executor agent is replaced with market watcher and data analyst agents to gather and plot the stock price data.

#### The task!

In [3]:
var task = """
Write a blogpost about the stock price performance of Nvidia in the past month.
Today's date is 2024-04-23.
""";

#### Build a group chat
This group chat will include these agents:

- User_proxy: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Market_Watcher: to gather the stock price data.
- Data_Analyst: to plot the stock price data.
- Writer: to write the report.

In [4]:
var user = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "user",
    modelName: openAIModel,
    systemMessage: """
    Give the task, and send instructions to writer to refine the blog post.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [5]:
var planner = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Planner",
    modelName: openAIModel,
    systemMessage: """
    You are Planner.
    Given a task, please determine what step is needed to complete the task.
    Please only suggest steps that can be done by others.
    After each step is done by others, check the progress and instruct the remaining steps.
    If a step fails, try to work around it.
    If the task is completed, say 'task completed'.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [7]:
var workingDirectory = Path.Combine(Directory.GetCurrentDirectory(), "work");
if (!Directory.Exists(workingDirectory))
{
    Directory.CreateDirectory(workingDirectory);
}

Create market tools to gather and plot the stock price data. The data analyst will plot the stock price data and the writer will write the report.

In [14]:
public class MarketTools
{
    private string workDir;

    public MarketTools(string workDir)
    {
        this.workDir = workDir;
    }

    /// <summary>
    /// Retrieve stock price data for a given symbol and date range.
    /// </summary>
    /// <param name="symbol">stock symbol.</param>
    /// <param name="from">from date, in the format of YYYY-MM-dd</param>
    /// <param name="to">to date, in the format of YYYY-MM-dd</param>
    /// <returns></returns>
    [KernelFunction]
    [Description("Retrieve stock price data for a given symbol and date range.")]
    public async Task<string> RetrieveStockPrice(
        [Description("stock symbol")]string symbol,
        [Description("from data, in the format of YYYY-MM-dd")]string from, 
        [Description("to date, in the format of YYYY-MM-dd")]string to)
    {
        var sb = new StringBuilder();
        sb.AppendLine($"Retrieving stock price data for {symbol} from {from} to {to}...");
        // Retrieve stock price data from the database
        var stockData = """
            2024-03-25    950.020020
            2024-03-26    925.609985
            2024-03-27    902.500000
            2024-03-28    903.559998
            2024-04-01    903.630005
            2024-04-02    894.520020
            2024-04-03    889.640015
            2024-04-04    859.049988
            2024-04-05    880.080017
            2024-04-08    871.330017
            2024-04-09    853.539978
            2024-04-10    870.390015
            2024-04-11    906.159973
            2024-04-12    881.859985
            2024-04-15    860.010010
            2024-04-16    874.150024
            2024-04-17    840.349976
            2024-04-18    846.710022
            2024-04-19    762.000000
            2024-04-22    795.179993
            """;

        sb.AppendLine(stockData);

        return sb.ToString();
    }

    /// <summary>
    /// Plot candle line chart for stock price data.
    /// </summary>
    /// <param name="symbol">stock symbol.</param>
    /// <param name="close">close price.</param>
    /// <param name="date">date, in the format of YYYY-MM-dd</param>
    [KernelFunction]
    [Description("Plot candle line chart for stock price data.")]
    public async Task<string> PlotStockLineChart(
        [Description("stock symbol")]string symbol,
        [Description("close price")]float[] close,
        [Description("date, in the format of YYYY-MM-dd")]string[] date)
    {
        var title = $"Stock Price Performance of {symbol} from {date[0]} to {date[^1]}";

        // copy img/nvidia-plot.png to workDir/{title}.png
        var targetPath = Path.Combine(workDir, $"{title}.png");
        var imgPath = Path.Combine("img", "nvidia-plot.png");
        File.Copy(imgPath, targetPath, true);

        var reply = $"""
            Stock price data has been plotted.
            You can view the chart at {targetPath}
            """;

        return reply;
    }
}

var kernel = new Kernel();
var marketTools = new MarketTools(workingDirectory);
var marketToolsPlugin = kernel.ImportPluginFromObject(marketTools);
var marketToolsPluginMiddleware = new KernelPluginMiddleware(kernel, marketToolsPlugin);

In [15]:
var marketWatcher = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Market_Watcher",
    modelName: openAIModel,
    systemMessage: """
    You are Market Watcher. You can gather stock price data.
    """)
    .RegisterMessageConnector()
    .RegisterMiddleware(marketToolsPluginMiddleware)
    .RegisterPrintMessage();

In [16]:
var dataAnalyst = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Data_Analyst",
    modelName: openAIModel,
    systemMessage: """
    You are Data Analyst. You can plot stock price data.
    """)
    .RegisterMessageConnector()
    .RegisterMiddleware(marketToolsPluginMiddleware)
    .RegisterPrintMessage();

In [17]:
var writer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Writer",
    modelName: openAIModel,
    systemMessage: """
    Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block.
    You take feedback from the admin and refine your blog.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

#### Define the group chat

Create a group chat admin to orchestrate the group chat

The admin will be responsible for selecting the next speaker

It will not directly participate in the conversation

In [18]:
var groupChatAdminAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Admin",
    modelName: openAIModel)
    .RegisterMessageConnector();

Add a speaker selection policy using workflow

In [19]:
var workflow = new Graph();

// user <=> planner
workflow.AddTransition(Transition.Create(user, planner));
workflow.AddTransition(Transition.Create(planner, user));

// planner <=> marketWatcher
workflow.AddTransition(Transition.Create(planner, marketWatcher));
workflow.AddTransition(Transition.Create(marketWatcher, planner));

// planner <=> dataAnalyst
workflow.AddTransition(Transition.Create(planner, dataAnalyst));
workflow.AddTransition(Transition.Create(dataAnalyst, planner));

// planner <=> writer
workflow.AddTransition(Transition.Create(planner, writer));
workflow.AddTransition(Transition.Create(writer, planner));

Create a group chat

In [23]:
var groupChat = new GroupChat(
    admin: groupChatAdminAgent,
    workflow: workflow,
    members: [user, planner, marketWatcher, dataAnalyst, writer]);

// Add agent self-description as inital messages.
groupChat.AddInitializeMessage(new TextMessage(Role.Assistant, "I am market watcher. I can gather stock price data.", from: marketWatcher.Name));
groupChat.AddInitializeMessage(new TextMessage(Role.Assistant, "I am data analyst. I can plot stock price data.", from: dataAnalyst.Name));
groupChat.AddInitializeMessage(new TextMessage(Role.Assistant, "I am writer. I can write blogs in markdown format.", from: writer.Name));

start the group chat!

In [26]:
var chatHistory = new List<IMessage>()
{
    new TextMessage(Role.User, task, from: user.Name),
};

var maxTurn = 20;
while (maxTurn > 0)
{
    var replies = await groupChat.CallAsync(chatHistory, maxRound: 1);
    var reply = replies.Last();
    chatHistory.Add(reply);
    maxTurn--;

    if (reply.GetContent()?.ToLower().Contains("task completed") is true
     && replies.Where(c => c.From == writer.Name).Any())
    {
        break;
    }
}

from: Planner
1. Market_Watcher: Gather Nvidia's stock price data for the past month up to today's date (2024-04-23).
2. Data_Analyst: Plot the stock price data for Nvidia for the past month to analyze its performance.
3. Writer: Write a blog post in markdown format discussing Nvidia's stock price performance in the past month based on the analysis.

Let's start with step 1. Market_Watcher, please gather Nvidia's stock price data for the past month up to today's date (2024-04-23).

AggregateMessage from Market_Watcher
--------------------
ToolCallMessage:
ToolCallMessage from Market_Watcher
--------------------
- RetrieveStockPrice: {"symbol":"NVDA","from":"2024-03-23","to":"2024-04-23"}
--------------------

ToolCallResultMessage:
ToolCallResultMessage from Market_Watcher
--------------------
- RetrieveStockPrice: Retrieving stock price data for NVDA from 2024-03-23 to 2024-04-23...
2024-03-25    950.020020
2024-03-26    925.609985
2024-03-27    902.500000
2024-03-28    903.559998
202

Error: Azure.RequestFailedException: The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.
Status: 500 (Internal Server Error)

Content:
{
  "error": {
    "message": "The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.",
    "type": "model_error",
    "param": null,
    "code": null
  }
}

Headers:
Date: Sat, 20 Jul 2024 00:19:17 GMT
Connection: keep-alive
openai-organization: REDACTED
openai-processing-ms: REDACTED
openai-version: REDACTED
Strict-Transport-Security: REDACTED
x-ratelimit-limit-requests: REDACTED
x-ratelimit-limit-tokens: REDACTED
x-ratelimit-remaining-requests: REDACTED
x-ratelimit-remaining-tokens: REDACTED
x-ratelimit-reset-requests: REDACTED
x-ratelimit-reset-tokens: REDACTED
X-Request-ID: REDACTED
CF-Cache-Status: REDACTED
Set-Cookie: REDACTED
X-Content-Type-Options: REDACTED
Server: cloudflare
CF-RAY: REDACTED
Alt-Svc: REDACTE